In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import numpy as np 
import pandas as pd 

In [28]:
df_train = pd.read_csv('data/train.csv', delimiter=',')
df_train.dataframeName = 'train.csv'
nRow, nCol = df_train.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 7352 rows and 563 columns


In [29]:
df_test = pd.read_csv('data/test.csv', delimiter=',')
df_test.dataframeName = 'test.csv'
nRow, nCol = df_test.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 2947 rows and 563 columns


In [4]:
acc_x1 = df_train['tBodyAcc-mean()-X']
acc_y1 = df_train['tBodyAcc-mean()-Y']
acc_z1 = df_train['tBodyAcc-mean()-Z']
sample_num_tr = df_train["subject"]

In [6]:
acc_x2 = df_test['tBodyAcc-mean()-X']
acc_y2 = df_test['tBodyAcc-mean()-Y']
acc_z2 = df_test['tBodyAcc-mean()-Z']

sample_num_te = df_test["subject"]
y=df_test["Activity"]

In [9]:
dummy_dict = {}
for i, v in enumerate(y.unique(),1):
    dummy_dict[v]=i
    

In [10]:
dummy_dict

{'STANDING': 1,
 'SITTING': 2,
 'LAYING': 3,
 'WALKING': 4,
 'WALKING_DOWNSTAIRS': 5,
 'WALKING_UPSTAIRS': 6}

In [11]:
lst = []
for v in y:
    lst.append([np.array(dummy_dict[v])])
    

In [13]:
## 人がいつ切り替わるかを記録する
ind = 0
sepa_lst_tr = []
old_num=1
while True:
    if len(sample_num_tr)== ind:
        sepa_lst_tr.append(ind)
        break
    if not old_num  == sample_num_tr[ind]:
        old_num=sample_num_tr[ind]
        sepa_lst_tr.append(ind)
        ind+=1
        continue
    ind+=1

In [15]:
## 各人のラベルごとにわける

ll_tr=[]
old_sepa =0
lst_tr = []
memo =[]
for sepa in sepa_lst_tr:
    memo =[]
    sepa_df=df_train.loc[old_sepa:sepa]
    old_ac = sepa_df["Activity"].loc[old_sepa]
    sub = sepa_df["subject"].loc[old_sepa]
    ind = old_sepa
    old_sepa=sepa
    c=0
    j=0
    old_ind =old_sepa
    while True:
        if c ==len(sepa_df)or( sepa_df["subject"].loc[ind]!=sub):
            break
        
        if not old_ac ==  sepa_df["Activity"].loc[ind]:
            old_ac=  sepa_df["Activity"].loc[ind]
            memo.append(ind)
    
            old_ind = ind
            # print(sepa_df["Activity"].loc[ind])
            j+=1
            
            continue
        ind+=1
        c+=1
        # j+=1
    lst_tr.append(memo)
    old_sepa+=1
    

In [16]:
## 人がいつ切り替わるかを記録する
ind = 0
sepa_lst_te = []
old_num=2
while True:
    if len(sample_num_te)== ind:
        sepa_lst_te.append(ind)
        break
    if not old_num  == sample_num_te[ind]:
        old_num=sample_num_te[ind]
        sepa_lst_te.append(ind)
        ind+=1
        continue
    ind+=1


## 各人のラベルごとにわける

old_sepa =0
lst_te = []
memo =[]
for sepa in sepa_lst_te:
    memo =[]
    sepa_df=df_test.loc[old_sepa:sepa]
    old_ac = sepa_df["Activity"].loc[old_sepa]
    sub = sepa_df["subject"].loc[old_sepa]
    ind = old_sepa
    old_sepa=sepa
    c=0
    j=0
    old_ind =old_sepa
    while True:
        if c ==len(sepa_df)or( sepa_df["subject"].loc[ind]!=sub):
            break
        
        if not old_ac ==  sepa_df["Activity"].loc[ind]:
            old_ac=  sepa_df["Activity"].loc[ind]
            memo.append(ind)
            old_ind = ind
            # print(sepa_df["Activity"].loc[ind])
            j+=1
            
            continue
        ind+=1
        c+=1
        # j+=1
    lst_te.append(memo)
    old_sepa+=1
    

In [18]:
sp_lst=[]
y_v_train =[]
x_v_train=[]
ol=0
for v in lst_tr:
    sp_lst=[]
    for v1 in v:
        d =df_train.loc[ol:v1]
        x = d['tBodyAcc-mean()-X']
        y = d['tBodyAcc-mean()-Y']
        z = d['tBodyAcc-mean()-Z']
        fft_x = np.fft.fft(x)
        fft_y = np.fft.fft(y)
        fft_z = np.fft.fft(z)
        v2 = d["Activity"].loc[ol]
        y_v_train.append(np.array(dummy_dict[v2]))
        ol=v1
        x_v_train.append((np.array([fft_x,fft_y,fft_z])))        

In [19]:
y_v_test =[]
x_v_test =[]
ol=0
for v in lst_te:
    sp_lst=[]
    for v1 in v:
        d =df_test.loc[ol:v1]
        x = d['tBodyAcc-mean()-X']
        y = d['tBodyAcc-mean()-Y']
        z = d['tBodyAcc-mean()-Z']
        v2 = d["Activity"].loc[ol]
        y_v_test.append(np.array(dummy_dict[v2]))
        ol=v1
        x_v_test.append((np.array([fft_x,fft_y,fft_z])))     

In [20]:
fs = 50  # サンプリング周波数 (Hz)

def apply_fft(signal, fs):
    freq = np.fft.fftfreq(len(signal), d=1/fs)
    fft_result = np.fft.fft(signal)
    return freq, fft_result

def filter_freq(freq, fft_result, min_, max_):
    pass_ = (freq >= min_) & (freq <= max_)
    filtered_fft = fft_result * pass_
    filtered = np.fft.ifft(filtered_fft)
    return filtered.real

def smooth_padding(signal, pad_length):
    """
    スムーズにゼロに遷移するパディングを追加する。
    """
    taper = np.linspace(signal[-1], 0, pad_length)  # スムーズにゼロに減少
    return np.concatenate([signal, taper])

def pad_to_length(data, target_length):
    current_length = len(data)
    if current_length < target_length:
        pad_length = target_length - current_length
        return smooth_padding(data, pad_length)
    return data[:target_length]

def process_signal(acc, fs, target_length):
    
    freq, fft = apply_fft(acc, fs)
    low = filter_freq(freq, fft, 0 ,5)
    middle = filter_freq(freq, fft,5, 10)
    mid = filter_freq(freq, fft, 10, 15)
    midd = filter_freq(freq, fft, 15, 20)
    high = filter_freq(freq, fft, 20, 25)
    # print(freq)
    return (
        pad_to_length(low, target_length),
        pad_to_length(middle, target_length),
        pad_to_length(mid, target_length),
        pad_to_length(midd, target_length),
        pad_to_length(high, target_length),
    )

x_v11 = []
x_v12 = []
x_v13 = []
x_v14=[]
x_v15=[]

for v in x_v_train:
    acc_x, acc_y, acc_z = v
    # 全体のターゲット長を決定
    target_length = max(len(acc_x), len(acc_y), len(acc_z))

    low_x, middle_x,mid_x,midd_x, high_x = process_signal(acc_x, fs, target_length)
    low_y, middle_y,mid_y,midd_y, high_y = process_signal(acc_y, fs, target_length)
    low_z, middle_z,mid_z,midd_z, high_z = process_signal(acc_z, fs, target_length)

    x_v11.append(np.array([low_x, low_y, low_z]))
    x_v13.append(np.array([middle_x, middle_y, middle_z]))
    x_v14.append(np.array([mid_x, mid_y, mid_z]))
    x_v15.append(np.array([midd_x, midd_y, midd_z]))
    x_v12.append(np.array([high_x, high_y, high_z]))


# わけたやつを基にもどすところ上からlow,middle,mid,midd,high

n = []
for i in range(len(x_v_train)):
    m=[]
    m2=[]
    m3=[]
    m4=[]
    m5=[]
    for k in range(len(x_v11[i][0])):
        m.append([x_v11[i][0][k],x_v11[i][1][k],x_v11[i][2][k]])
        
        
    for k in range(len(x_v13[i][0])):
        m3.append([x_v13[i][0][k],x_v13[i][1][k],x_v13[i][2][k]],)
        
    
    for k in range(len(x_v14[i][0])):
        m4.append([x_v14[i][0][k],x_v14[i][1][k],x_v14[i][2][k]],)
    
    
    for k in range(len(x_v15[i][0])):
        m5.append([x_v15[i][0][k],x_v15[i][1][k],x_v15[i][2][k]],)
    
        
    for k in range(len(x_v12[i][0])):
        m2.append([x_v12[i][0][k],x_v12[i][1][k],x_v12[i][2][k]],)
    n.append([np.array(m),np.array(m2)])

num_classes=6

X_train=n
Y_train = np.array(y_v_train)-1
Y_train_one_hot = np.eye(num_classes)[Y_train]

In [21]:
x_v21 = []
x_v22 = []
x_v23 = []
x_v24=[]
x_v25=[]

for v in x_v_test:
    acc_x, acc_y, acc_z = v
    # 全体のターゲット長を決定
    target_length = max(len(acc_x), len(acc_y), len(acc_z))

    low_x, middle_x,mid_x,midd_x, high_x = process_signal(acc_x, fs, target_length)
    low_y, middle_y,mid_y,midd_y, high_y = process_signal(acc_y, fs, target_length)
    low_z, middle_z,mid_z,midd_z, high_z = process_signal(acc_z, fs, target_length)

    x_v21.append(np.array([low_x, low_y, low_z]))
    x_v23.append(np.array([middle_x, middle_y, middle_z]))
    x_v24.append(np.array([mid_x, mid_y, mid_z]))
    x_v25.append(np.array([midd_x, midd_y, midd_z]))
    x_v22.append(np.array([high_x, high_y, high_z]))

n1 = []
for i in range(len(x_v_test)):
    m=[]
    m2=[]
    m3=[]
    m4=[]
    m5=[]
    for k in range(len(x_v21[i][0])):
        m.append([x_v21[i][0][k],x_v21[i][1][k],x_v21[i][2][k]])
        
        
    for k in range(len(x_v23[i][0])):
        m3.append([x_v23[i][0][k],x_v23[i][1][k],x_v23[i][2][k]],)
        
    
    for k in range(len(x_v24[i][0])):
        m4.append([x_v24[i][0][k],x_v24[i][1][k],x_v24[i][2][k]],)
    
    
    for k in range(len(x_v25[i][0])):
        m5.append([x_v25[i][0][k],x_v25[i][1][k],x_v25[i][2][k]],)
    
        
    for k in range(len(x_v22[i][0])):
        m2.append([x_v22[i][0][k],x_v22[i][1][k],x_v22[i][2][k]],)
    n1.append([np.array(m),np.array(m2)])
    
X_test=n1
Y_test = np.array(y_v_test)-1
Y_test_one_hot = np.eye(num_classes)[Y_test]

In [22]:
import numpy as np

class ParallelDeepESN:
    def __init__(self, input_size, reservoir_sizes, num_classes, sparsity=0.1, spectral_radius=0.95):
        """
        Parallel Deep ESNモデルの初期化
        :param input_size: 各周波数帯データの入力次元
        :param reservoir_sizes: 各リザバーのサイズ（各周波数帯ごと: [size1, size2, ..., sizeM]）
        :param num_classes: 出力のクラス数
        :param sparsity: リザバーのスパース性（結合の密度）
        :param spectral_radius: スペクトル半径（リザバーの安定性を制御）
        """
        self.num_processes = len(reservoir_sizes)  # 周波数帯の数（リザバー層数）
        self.reservoirs = []
        self.weights_input = []
        self.weights_reservoir = []

        # 各リザバー層の初期化
        for reservoir_size in reservoir_sizes:
            # 入力 -> リザバーの重み行列
            W_in = (np.random.rand(reservoir_size, input_size) - 0.5) * 1.0
            # リザバーの重み行列
            W_res = (np.random.rand(reservoir_size, reservoir_size) - 0.5) * 1.0
            W_res[np.random.rand(*W_res.shape) > sparsity] = 0  # スパース化
            eigvals = np.max(np.abs(np.real(np.linalg.eigvals(W_res))))  # 実数成分のみ使用
            W_res *= spectral_radius / eigvals  # スペクトル半径調整

            self.weights_input.append(W_in)
            self.weights_reservoir.append(W_res)
            self.reservoirs.append(np.zeros((reservoir_size, 1)))  # 初期状態（ゼロ）

        # 出力層の重み行列（学習対象）
        self.W_output = np.zeros((num_classes, sum(reservoir_sizes)))  # 総リザバー出力サイズ

    def forward(self, inputs_list):
        """
        各周波数帯ごとに独立したリザバーを更新
        :param inputs_list: 1つのサンプルにおける周波数帯データのリスト（[周波数帯1, 周波数帯2, ...]）
        :return: 統合されたリザバー状態ベクトル
        """
        all_reservoir_states = []
        for i, inputs in enumerate(inputs_list):
            W_in = self.weights_input[i]
            W_res = self.weights_reservoir[i]
            reservoir = self.reservoirs[i]

            reservoir_states = []  # 時系列ごとのリザバー状態を保存
            for t in range(inputs.shape[0]):  # 時系列を順に処理
                u_t = inputs[t].reshape(-1, 1)  # 列ベクトル化
                reservoir = np.tanh(np.dot(W_res, reservoir) + np.dot(W_in, u_t))
                reservoir = np.real(reservoir)  # 複素数成分を防ぐため実数部分のみを保持
                reservoir_states.append(reservoir)

            # 時系列データを時間方向に平均化（または他の統計量を使用可能）
            aggregated_reservoir = np.mean(np.hstack(reservoir_states), axis=1, keepdims=True)
            
            
            reservoir_states_stacked = np.hstack(reservoir_states)
            mean_reservoir = np.mean(reservoir_states_stacked, axis=1, keepdims=True)
            var_reservoir = np.var(reservoir_states_stacked, axis=1, keepdims=True)
            aggregated_reservoir = np.vstack([mean_reservoir, var_reservoir])

            
            all_reservoir_states.append(aggregated_reservoir)

        # 各周波数帯のリザバー状態を連結して返す
        return np.vstack(all_reservoir_states)

    def train(self, X, targets_one_hot, regularization=1e-12):
        """
        出力層の重み W_output を学習（リッジ回帰を使用）
        :param X: 入力データ（各サンプルの [[周波数帯1, 周波数帯2, ...], ...]）
        :param targets_one_hot: one-hotエンコードされたターゲット（例: クラス数 x サンプル数）
        :param regularization: 正則化項
        """
        reservoir_states = []
        for inputs_list in X:
            reservoir_states.append(self.forward(inputs_list))

        # 状態行列 X を構築
        X_concat = np.hstack(reservoir_states)  # 各サンプルの最終リザバー状態を列として並べる
        Y = np.array(targets_one_hot).T  # ワンホットターゲットを転置して (num_classes, num_samples)

        # エラー回避のための形状チェック
        if X_concat.shape[1] < X_concat.shape[0]:
            print("警告: サンプル数がリザバー状態の次元より少ないため、逆行列計算が不安定になる可能性があります。")
            # 正則化を増やして安定性を確保
            regularization = max(regularization, 1e-2)
        # regularization=1e-8
        # regularization = 1e-4 * np.linalg.norm(np.dot(X_concat, X_concat.T))
        # X_concat = (X_concat - np.min(X_concat)) / (np.max(X_concat) - np.min(X_concat))
        # X_concat = (X_concat - np.min(X_concat)) / (np.max(X_concat) - np.min(X_concat))
        # X_concat = (X_concat - np.mean(X_concat, axis=1, keepdims=True)) / np.std(X_concat, axis=1, keepdims=True)


        # リッジ回帰で W_output を計算
        self.W_output = np.dot(
            np.dot(Y, X_concat.T),
            np.linalg.inv(np.dot(X_concat, X_concat.T) + regularization * np.eye(X_concat.shape[0]))
        )

        # 訓練データでのクロスエントロピーを計算
   

        logits = np.real(np.dot(self.W_output, X_concat))  # 実数成分のみを保持
        softmax_probs = np.exp(logits) / np.sum(np.exp(logits), axis=0)
        cross_entropy_loss = -np.sum(Y * np.log(softmax_probs + 1e-12)) / Y.shape[1]
        print(f"Training Cross-Entropy Loss: {cross_entropy_loss:.4f}")



    def predict(self, X, targets_one_hot=None):
        """
        複数の入力データ群を用いてクラスを予測し、必要に応じてクロスエントロピー損失を計算
        :param X: 入力データ（各サンプルの [[周波数帯1, 周波数帯2, ...], ...] リスト）
        :param targets_one_hot: one-hotエンコードされたターゲット（例: クラス数 x サンプル数）。指定された場合にクロスエントロピーを計算。
        :return: 各サンプルの予測クラス、必要に応じてクロスエントロピー損失
        """
        predictions = []
        reservoir_states = []

        for inputs_list in X:
            final_state = self.forward(inputs_list)
            logits = np.real(np.dot(self.W_output, final_state))  # 実数成分のみを保持
            predicted_class = np.argmax(logits, axis=0)  # クラスを予測
            predictions.append(predicted_class)
            reservoir_states.append(final_state)

        # 必要に応じてクロスエントロピーを計算
        cross_entropy_loss = None
        if targets_one_hot is not None:
            # 状態行列を再構築
            X_concat = np.hstack(reservoir_states)  # 各サンプルのリザバー状態を結合
            Y = np.array(targets_one_hot).T  # ワンホットターゲットを転置 (num_classes, num_samples)

            # ロジット（予測値の前処理）
            logits = np.real(np.dot(self.W_output, X_concat))  # 実数成分のみを保持
            softmax_probs = np.exp(logits) / np.sum(np.exp(logits), axis=0)  # ソフトマックス
            cross_entropy_loss = -np.sum(Y * np.log(softmax_probs + 1e-12)) / Y.shape[1]  # クロスエントロピー計算
            print(f"Prediction Cross-Entropy Loss: {cross_entropy_loss:.4f}")

        return predictions, cross_entropy_loss

In [23]:
# サンプル数と設定
reservoir_sizes = [25,25,25,25,25]  # 工程数をnに増加
num_classes = 6
input_size = 3

# モデルの初期化と学習
model = ParallelDeepESN(input_size=input_size, reservoir_sizes=reservoir_sizes, num_classes=num_classes,
                       sparsity=0.8, spectral_radius=0.95)
model.train(X_train, Y_train_one_hot)

predictions, loss = model.predict(X_test, targets_one_hot=Y_test_one_hot)

Training Cross-Entropy Loss: 1.2988
Prediction Cross-Entropy Loss: 1.7945


In [24]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

t =[]
for i in predictions:
    t.append(i[0])
t=np.array(t)
true_labels = Y_test
predicted_labels = t

# 混同行列の計算
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("混同行列:\n", conf_matrix)

# F1スコア（マクロ平均）
f1 = f1_score(true_labels, predicted_labels, average='macro')
print("F1スコア (マクロ平均):", f1)

# 詳細なレポート

report = classification_report(true_labels, predicted_labels, target_names=["Standing",
"Sitting","Laying","Walking","Walking_downstairs", "Waling_Upstairs"])
print("\n分類レポート:\n", report)

混同行列:
 [[ 0  0  0  0 10  0]
 [ 0  0  0  0 18  0]
 [ 0  0  0  0 18  0]
 [ 0  0  0  0 18  0]
 [ 0  0  0  0 25  0]
 [ 0  0  0  0 22  0]]
F1スコア (マクロ平均): 0.06127450980392157

分類レポート:
                     precision    recall  f1-score   support

          Standing       0.00      0.00      0.00        10
           Sitting       0.00      0.00      0.00        18
            Laying       0.00      0.00      0.00        18
           Walking       0.00      0.00      0.00        18
Walking_downstairs       0.23      1.00      0.37        25
   Waling_Upstairs       0.00      0.00      0.00        22

          accuracy                           0.23       111
         macro avg       0.04      0.17      0.06       111
      weighted avg       0.05      0.23      0.08       111



C:\Users\admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
